In [8]:
import numpy as np
np.random.seed(20)
x = np.arange(20)
y = [x*2 + np.random.rand(1)*4 for x in range(20)]

In [9]:
x

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19])

In [10]:
y

[array([2.3525232]),
 array([5.59085491]),
 array([7.56612292]),
 array([9.26334991]),
 array([8.14355834]),
 array([12.76703033]),
 array([13.51472377]),
 array([16.07404378]),
 array([18.63180586]),
 array([18.77540087]),
 array([21.08926561]),
 array([24.87442373]),
 array([27.13201444]),
 array([29.40131056]),
 array([31.10097958]),
 array([30.14665723]),
 array([32.46677494]),
 array([37.0051228]),
 array([36.95687286]),
 array([39.01922406])]

In [11]:
x_reshape = x.reshape(-1,1)


In [13]:
x_reshape

array([[ 0],
       [ 1],
       [ 2],
       [ 3],
       [ 4],
       [ 5],
       [ 6],
       [ 7],
       [ 8],
       [ 9],
       [10],
       [11],
       [12],
       [13],
       [14],
       [15],
       [16],
       [17],
       [18],
       [19]])

In [14]:
from sklearn.linear_model import LinearRegression
linear = LinearRegression()
linear.fit(x_reshape, y)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/base.py:509: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  linalg.lstsq(X, y)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [15]:
class Stats:
    
    def __init__(self, X, y, model):
        self.data = X
        self.target = y
        self.model = model
        ## degrees of freedom population dep. variable variance
        self._dft = X.shape[0] - 1   
        ## degrees of freedom population error variance
        self._dfe = X.shape[0] - X.shape[1] - 1  
    
    def sse(self):
        '''returns sum of squared errors (model vs actual)'''
        squared_errors = (self.target - self.model.predict(self.data)) ** 2
        return np.sum(squared_errors)
        
    def sst(self):
        '''returns total sum of squared errors (actual vs avg(actual))'''
        avg_y = np.mean(self.target)
        squared_errors = (self.target - avg_y) ** 2
        return np.sum(squared_errors)
    
    def r_squared(self):
        '''returns calculated value of r^2'''
        return 1 - self.sse()/self.sst()
    
    def adj_r_squared(self):
        '''returns calculated value of adjusted r^2'''
        return 1 - (self.sse()/self._dfe) / (self.sst()/self._dft)

In [16]:
def pretty_print_stats(stats_obj):
    '''returns report of statistics for a given model object'''
    items = ( ('sse:', stats_obj.sse()), ('sst:', stats_obj.sst()), 
             ('r^2:', stats_obj.r_squared()), ('adj_r^2:', stats_obj.adj_r_squared()) )
    for item in items:
        print('{0:8} {1:.4f}'.format(item[0], item[1]))

In [17]:
s1 = Stats(x_reshape, y, linear)
pretty_print_stats(s1)

sse:     24.3975
sst:     2502.9934
r^2:     0.9903
adj_r^2: 0.9897


In [18]:
#Non-Linear

In [19]:
y_nonlinear = [x**3 + np.random.rand(1)*10 for x in range(20)]
nonlinear = LinearRegression()
nonlinear.fit(x_reshape, y_nonlinear)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [22]:
#Normality test
#here it turns out the residuals for the nonlinear function are Normally distributed as well.


residuals_nlinear = y_nonlinear - nonlinear.predict(x_reshape)
from scipy.stats import normaltest
normaltest(residuals_nlinear)

NormaltestResult(statistic=array([2.20019716]), pvalue=array([0.33283827]))

In [23]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

poly = Pipeline([('poly', PolynomialFeatures(degree=3)),
                  ('linear', LinearRegression(fit_intercept=False))])
poly.fit(x_reshape, y_nonlinear)

Pipeline(memory=None,
     steps=[('poly', PolynomialFeatures(degree=3, include_bias=True, interaction_only=False)), ('linear', LinearRegression(copy_X=True, fit_intercept=False, n_jobs=1, normalize=False))])

In [26]:
#high Leverage
np.random.seed(20)
x = np.arange(20)
y_linear_leverage = [x*2 + np.random.rand(1)*4 for x in range(20)]
y = [x*2 + np.random.rand(1)*4 for x in range(20)]
y_outlier = y.copy()
y_outlier[8] = np.array([38])  ## insert outlier
y_linear_leverage[18] = np.array([55])  ## high-leverage point
y_linear_leverage[19] = np.array([58])  ## high-leverage point
x_reshape = x.reshape(-1,1)
linear_leverage = LinearRegression()
linear_leverage.fit(x_reshape, y_linear_leverage)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [27]:
normaltest(y_outlier-linear_leverage.predict(x_reshape))


NormaltestResult(statistic=array([25.40983968]), pvalue=array([3.03615137e-06]))

In [28]:
np.random.seed(20)
x = np.arange(20)
y_homo = [x*2 + np.random.rand(1) for x in range(20)]  ## homoscedastic error
y_hetero = [x*2 + np.random.rand(1)*2*x for x in range(20)]  ## heteroscedastic error

In [29]:
x_reshape = x.reshape(-1,1)
#Fit Model
linear_homo = LinearRegression()
linear_homo.fit(x_reshape, y_homo)

linear_hetero = LinearRegression()
linear_hetero.fit(x_reshape, y_hetero)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [31]:
# homoscedastic data
normaltest(y_homo-linear_homo.predict(x_reshape))


NormaltestResult(statistic=array([1.71234546]), pvalue=array([0.42478474]))

In [32]:

# heteroscedastic data
normaltest(y_hetero-linear_hetero.predict(x_reshape))

NormaltestResult(statistic=array([1.04126656]), pvalue=array([0.59414417]))

In [33]:
y_hetero_log = np.log10(np.array(y_hetero) + 1e1)
x_reshape_log = np.log10(np.array(x_reshape) + 1e1)

linear_hetero_log = LinearRegression()
linear_hetero_log.fit(x_reshape, y_hetero_log)

linear_hetero_log_log = LinearRegression()
linear_hetero_log_log.fit(x_reshape_log, y_hetero_log)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [34]:
normaltest(y_hetero_log - linear_hetero_log.predict(x_reshape))


NormaltestResult(statistic=array([0.96954754]), pvalue=array([0.6158365]))

In [35]:
np.random.seed(20)
x = np.arange(20)
y_uncorr = [2*x + np.random.rand(1) for x in range(20)]
y_corr = np.sin(x)
#Reshape x
x_reshape = x.reshape(-1,1)
#Fit Model
linear_uncorr = LinearRegression()
linear_uncorr.fit(x_reshape, y_uncorr)

linear_corr = LinearRegression()
linear_corr.fit(x_reshape, y_corr)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)